In [7]:
import warnings
warnings.filterwarnings('ignore')
from nltk.stem import PorterStemmer
import pandas as pd
from wordcloud import STOPWORDS,WordCloud

In [3]:
# It helps us to remove common morphological and inflexional endings from words in English
stemmer = PorterStemmer()

In [4]:
def get_stem_word(text: str) -> str:
    
    stemmed_words = []
    sent_list = text.split()
    
    for word in sent_list:
        try:
            stemmed_words.append(stemmer.stem(word)) 
        except:
             stemmed_words.append(word)
    return ' '.join(stemmed_words)

In [32]:
# Read the data
df = pd.read_csv('cleaned_fintech_data.csv')

Sentiment Analyisis

In [36]:
# inspecting the columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5621 entries, 0 to 5620
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              5620 non-null   float64
 1   created_at              5621 non-null   object 
 2   source                  5621 non-null   object 
 3   original_text           5621 non-null   object 
 4   clean_text              5617 non-null   object 
 5   sentiment               5621 non-null   object 
 6   polarity                5621 non-null   object 
 7   subjectivity            5621 non-null   object 
 8   lang                    5621 non-null   object 
 9   favorite_count          5621 non-null   object 
 10  retweet_count           5621 non-null   object 
 11  original_author         5621 non-null   object 
 12  screen_count            5621 non-null   object 
 13  followers_count         5621 non-null   object 
 14  friends_count           5621 non-null   

In [16]:
df['polarity'].value_counts()

 0.000000    1829
-0.750000     501
 0.500000     210
 0.133333     202
 0.337500     189
             ... 
 0.096591       1
 0.126667       1
 0.208333       1
-0.054167       1
 0.154107       1
Name: polarity, Length: 380, dtype: int64

In [35]:
# checking if there are null values
text_null_sum = df['clean_text'].isnull().sum()
polarity_null_sum = df['polarity'].isnull().sum()

print(f'sum of null values in clean_text column: {text_null_sum}')
print(f'sum of null values in polarity cloumn: {polarity_null_sum}')


sum of null values in clean_text column: 4
sum of null values in polarity cloumn: 0


In [34]:
df['clean_text'].fillna('')

0       Giving forth life becoming burden Kenya This m...
1       Teenmaar crPanja crGabbarsingh cr Khaleja Kuda...
2       Rei chintu lo Vachina Ad Nizam ne lo kottaru f...
3       Today World Day Combat Restoring degraded land...
4       Hearing say 's confident vaccines delivered li...
                              ...                        
5616    Lambie voted AGAINST Medivac Carbon tax protec...
5617    Idhi bathuku Most Day1 Records In Nizam Non BB...
5618    welkin moon giveaway bc finally money -u got t...
5619    Manam edi chesina Daaniki kuda elevation istad...
5620    This first time I 've seen Government media le...
Name: clean_text, Length: 5621, dtype: object

Data preparation

In [26]:
# creating cleanedTweet dataframe
cleanedTweet = pd.DataFrame(columns=['clean_text','polarity'])

cleanedTweet['clean_text'] = df['clean_text']
cleanedTweet['polarity'] = df['polarity']
cleanedTweet.dropna(inplace=True)

In [22]:
cleanedTweet

,clean_text,polarity
0,"africa is ""in the midst of a full-blown third ...",0.166667
1,dr moeti is head of who in africa and one of t...,0.133333
2,thank you for creating this amazing campaign ...,0.316667
3,former pfizer vp and virologist dr michael yea...,0.086111
4,i think it s important that we don t sell cova...,0.280000
...,...,...
6527,rt : zweli please just release the graphic des...,0.000000
6528,former pfizer vp and virologist dr michael yea...,0.086111
6529,respectfully veterinarians have to have a broa...,0.281250
6530,"""africa needs millions more doses here &amp; n...",0.125000


In [ ]:


# takes polarity and returns a string either postive, negative or neutral

def text_category(p: float) -> str:
    if p > 0:
        return "positive"
    elif p == 0:
        return "neutral"
    else:
        return "negative"

